In [9]:
import numpy as np
import cv2
import sys
import time

### 9.3.5 프로그래밍 실습: YOLO v3으로 물체 검출

In [8]:
def construct_yolo_v3():
    f = open("./saved_model/coco_names.txt", 'r')
    class_names = [line.strip() for line in f.readlines()]
    
    model =cv2.dnn.readNet("./saved_model/yolov3.weights", "./saved_model/yolov3.cfg")
    layer_names = model.getLayerNames()
    out_layers = [layer_names[i-1] for i in model.getUnconnectedOutLayers()]
    
    return model, out_layers, class_names


def yolo_detect(img, yolo_model, out_layers):
    height, width = img.shape[0], img.shape[1]
    test_img = cv2.dnn.blobFromImage(img, 1.0/256, (448, 448), (0, 0, 0), swapRB=True)
    yolo_model.setInput(test_img)
    output3 = yolo_model.forward(out_layers)
    
    box, conf, id = [], [], []
    for output in output3:
        for vec85 in output:
            scores = vec85[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                centerx, centery = int(vec85[0]*width), int(vec85[1]*height)
                w, h = int(vec85[2]*width), int(vec85[3]*height)
                x, y = int(centerx - w/2), int(centery - h/2)
                box.append([x, y, x+w, y+h])
                conf.append(float(confidence))
                id.append(class_id)
    ind = cv2.dnn.NMSBoxes(box, conf, 0.5, 0.4)
    objects = [box[i] + [conf[i]] + [id[i]] for i in range(len(box)) if i in ind]
    return objects


model, out_layers, class_names = construct_yolo_v3()
colors = np.random.uniform(0, 255, size=(len(class_names), 3))

img = cv2.imread("./imgs/soccer.jpg")

res = yolo_detect(img, model, out_layers)

for i in range(len(res)):
    x1, y1, x2, y2, confidence, id = res[i]
    text = str(class_names[id]) + "%.3f" %confidence
    cv2.rectangle(img, (x1, y1), (x2, y2), colors[id], 2)
    cv2.putText(img, text, (x1, y1+30), cv2.FONT_HERSHEY_PLAIN, 1.5, colors[id], 2)
    
cv2.imshow("Object detection by YOLO v.3", img)
cv2.waitKey()
cv2.destroyAllWindows()

In [14]:
cap = cv2.VideoCapture("video/boy.mov")

start = time.time()
n_frame = 0
while True:
    ret, frame = cap.read()
    
    res = yolo_detect(frame, model, out_layers)
    
    for i in range(len(res)):
        x1, y1, x2, y2, confidence, id = res[i]
        text = str(class_names[id]) + "%.3f"%confidence
        cv2.rectangle(frame, (x1, y1), (x2, y2), colors[id], 2)
        cv2.putText(frame, text, (x1, y1+30), cv2.FONT_HERSHEY_PLAIN, 1.5, colors[id], 2)
    
    cv2.imshow("object detection from video by YOLO v.3", frame)
    n_frame += 1
    
    key = cv2.waitKey(1)
    if key == ord('q'): break

end = time.time()
print("처리한 프레임수 =", n_frame, ", 경과시간=", end-start, "\n초당 프레임 수=", n_frame/(end-start))

cap.release()
cv2.destroyAllWindows()

처리한 프레임수 = 23 , 경과시간= 6.642044544219971 
초당 프레임 수= 3.4627891828902326
